<a href="https://colab.research.google.com/github/ecaidav/NYC-ELA-Scores/blob/main/2023-NYC-NYS-ELA-Scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install altair-saver

In [2]:
import pandas as pd
import altair as alt
import altair_saver as saver
from vega_datasets import data
from textwrap import wrap

In [5]:
# chart 1: NYS ELA scores by NYC school district

ELA_district = pd.read_csv("https://data.cityofnewyork.us/resource/iebs-5yhr.csv?$query=SELECT%20report_category%2C%20geographic_subdivision%2C%20school_name%2C%20grade%2C%20year%2C%20category%2C%20number_tested%2C%20mean_scale_score%2C%20level_1%2C%20level_1_1%2C%20level_2%2C%20level_2_1%2C%20level_3%2C%20level_3_1%2C%20level_4%2C%20level_4_1%2C%20level_3_4%2C%20level_3_4_1%20WHERE%20(upper(%60report_category%60)%20LIKE%20'%25DISTRICT%25')%20AND%20((%60year%60%20%3D%20'2023')%20AND%20%60year%60%20IS%20NOT%20NULL)%20AND%20(upper(%60grade%60)%20LIKE%20'%253%25')%20AND%20(upper(%60category%60)%20LIKE%20'%25ALL%20STUDENTS%25')%20ORDER%20BY%20geographic_subdivision%20DESC")

ELA_district = ELA_district[['geographic_subdivision', 'level_1_1', 'level_2_1', 'level_3_1', 'level_4_1']]
ELA_district = ELA_district.rename(columns={
    'geographic_subdivision': 'District',
    'level_1_1': 'Level 1',
    'level_2_1': 'Level 2',
    'level_3_1': 'Level 3',
    'level_4_1': 'Level 4'
})

ELA_district['Level 3 + 4'] = ELA_district['Level 3'] + ELA_district['Level 4']
ELA_district= ELA_district.sort_values(by='Level 3 + 4', ascending=False).reset_index(drop=True)

ELA_district_long = ELA_district.melt(id_vars=['District'],
                                      value_vars=['Level 1', 'Level 2', 'Level 3', 'Level 4'],
                                      var_name='Level',
                                      value_name='Value')

district_scale = alt.Scale(
    domain=['Level 1', 'Level 2', 'Level 3', 'Level 4'],
    range=['#1e285f', '#7f9ccf', '#f6af42', '#7abb42']
)

district_color = alt.Color('Level:N', title=None, scale=district_scale)

district_click = alt.selection_multi(encodings=["y"])

district_base = alt.Chart(ELA_district_long).mark_bar(size=16, stroke='white', strokeWidth=1.5
).encode(
    x=alt.X('sum(Value):Q', stack='normalize', title=None, axis=None),
    y=alt.Y('District:N', sort=alt.EncodingSortField(field='Level 3 + 4', order='descending'), title=None, axis=alt.Axis(labelLimit=600)),
    tooltip=[alt.Tooltip('District:N', title='District'), alt.Tooltip('Level:N', title='Score'), alt.Tooltip('Value:N', title='%')]
)

district_bars = district_base.encode(
    detail='Level:N',
    color=alt.condition(district_click, district_color, alt.value("lightgray")),
).properties(
    width=550,
).add_selection(
    district_click
)

district_text = district_base.mark_text(align='right', dx=-3, dy=1, fontSize=12).encode(
    x=alt.X('Value:Q', stack='normalize'),
    y=alt.Y('District:N', sort=alt.EncodingSortField(field='Level 3 + 4', order='descending')),
    detail='Level:N',
    text=alt.Text('sum(Value):Q', format='.1f'),
    color=alt.condition(
        alt.datum.Level < 'Level 3',
        alt.value("white"),
        alt.value("black"),
    ),
    opacity=alt.condition('datum.Value==0', alt.value(0.0), alt.value(1.0))
)

district_chart = alt.layer(district_bars, district_text
).properties(
    width=900,
    height=600,
    title=alt.TitleParams(
      "Citywide",
      subtitle=wrap("Figure represents third grade scores on the 2023 NYS ELA assessment by New York City school district.", 109),
      anchor='start',
      orient='bottom',
      offset=10)
)

In [6]:
# chart 2: NYS ELA scores by District 9 school

ELA_school = pd.read_csv("https://data.cityofnewyork.us/resource/iebs-5yhr.csv?$query=SELECT%20report_category%2C%20geographic_subdivision%2C%20school_name%2C%20grade%2C%20year%2C%20category%2C%20number_tested%2C%20mean_scale_score%2C%20level_1%2C%20level_1_1%2C%20level_2%2C%20level_2_1%2C%20level_3%2C%20level_3_1%2C%20level_4%2C%20level_4_1%2C%20level_3_4%2C%20level_3_4_1%20WHERE%20(upper(%60report_category%60)%20LIKE%20'%25SCHOOL%25')%20AND%20((%60year%60%20%3D%20'2023')%20AND%20%60year%60%20IS%20NOT%20NULL)%20AND%20(upper(%60geographic_subdivision%60)%20LIKE%20'%2509X%25')%20AND%20(upper(%60grade%60)%20LIKE%20'%253%25')%20AND%20(upper(%60category%60)%20LIKE%20'%25ALL%20STUDENTS%25')%20ORDER%20BY%20report_category%20DESC")

ELA_school = ELA_school[['school_name', 'level_1_1', 'level_2_1', 'level_3_1', 'level_4_1']]
ELA_school = ELA_school.rename(columns={
    'school_name': 'District 9 School',
    'level_1_1': 'Level 1',
    'level_2_1': 'Level 2',
    'level_3_1': 'Level 3',
    'level_4_1': 'Level 4'
})

ELA_school['Level 3 + 4'] = ELA_school['Level 3'] + ELA_school['Level 4']
ELA_school= ELA_school.sort_values(by='Level 3 + 4', ascending=False).reset_index(drop=True)

ELA_school_long = ELA_school.melt(id_vars=['District 9 School'],
                                  value_vars=['Level 1', 'Level 2', 'Level 3', 'Level 4'],
                                  var_name='Level',
                                  value_name='Value')

school_scale = alt.Scale(
    domain=['Level 1', 'Level 2', 'Level 3', 'Level 4'],
    range=['#1e285f', '#7f9ccf', '#f6af42', '#7abb42']
)

school_color = alt.Color('Level:N', title=None, scale=school_scale)

school_click = alt.selection_multi(encodings=["y"])

school_base = alt.Chart(ELA_school_long).mark_bar(size=18, stroke='white', strokeWidth=1.5).encode(
    x=alt.X('sum(Value):Q', stack='normalize', title=None, axis=None),
    y=alt.Y('District 9 School:N', sort=alt.EncodingSortField(field='Level 3 + 4', order='descending'), title=None, axis=alt.Axis(labelLimit=600)),
    tooltip=[alt.Tooltip('District 9 School:N', title='School'), alt.Tooltip('Level:N', title='Score'), alt.Tooltip('Value:N', title='%')]
)

school_bars = school_base.encode(
    detail='Level:N',
    color=alt.condition(school_click, school_color, alt.value("lightgray")),
).properties(
    width=550,
).add_selection(
    school_click
)

school_text = school_base.mark_text(align='right', dx=-3, dy=1, fontSize=12).encode(
    x=alt.X('Value:Q', stack='normalize'),
    y=alt.Y('District 9 School:N', sort=alt.EncodingSortField(field='Level 3 + 4', order='descending')),
    detail='Level:N',
    text=alt.Text('sum(Value):Q', format='.1f'),
    color=alt.condition(
        alt.datum.Level < 'Level 3',
        alt.value("white"),
        alt.value("black"),
    ),
    opacity=alt.condition('datum.Value < 4', alt.value(0.0), alt.value(1.0))
)

school_chart = alt.layer(school_bars, school_text
).properties(
    width=900,
    height=600,
    title=alt.TitleParams(
      "District 9",
      subtitle=wrap("Figure represents third grade scores on the 2023 NYS ELA assessment by District 9 school.", 109),
      anchor='start',
      orient='bottom',
      align='left',
      offset=10)
)

In [7]:
# render charts side-by-side

full_chart = alt.hconcat(district_chart, school_chart).configure_legend(
    labelFontSize=12,
    titleFontSize=16,
    orient='top',
    labelBaseline='middle',
    symbolType='circle'
).configure_title(
    anchor='start',
    orient='bottom',
    offset=10
).configure_axis(
    labelFontSize=12,
    grid=False,
    ticks=False,
    labelPadding=15,
    titlePadding=15,
    orient='right'
).configure_view(
    stroke='white'
).properties(
    title=alt.TitleParams(
      "2023 NYC ELA Scores by % Level Achieved",
      subtitle=wrap("Students performing at Level 3 or higher are considered proficient in standards for their grade.", 109),
      anchor='start',
      orient='top',
      align='left',
      offset=10)
)

saver.save(full_chart, "NYC_ELA_Scores.html")

In [9]:
# render full chart

full_chart

alt.HConcatChart(...)